In [9]:
import requests

res = requests.get("https://gpt-index.readthedocs.io/en/latest/index.html")
res

<Response [200]>

In [12]:
import requests
from bs4 import BeautifulSoup

# Get the HTML content from the GPT Index documentation page
url = 'https://gpt-index.readthedocs.io/en/latest/index.html'
response = requests.get(url)
html_content = response.content

# Parse the HTML content with Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all the paragraphs on the page
paragraphs = soup.find_all('p')

# Store the text content of each paragraph in a list
content = []
for paragraph in paragraphs:
    content.append(paragraph.get_text())

# Print the list of content
print(content)

['Getting Started', 'Guides', 'Use Cases', 'Key Components', 'Reference', 'Gallery', 'LlamaIndex (GPT Index) is a project that provides a central interface to connect your LLM’s with external data.', 'Github: https://github.com/jerryjliu/llama_index', 'LlamaIndex: https://pypi.org/project/llama-index/.', 'GPT Index (duplicate): https://pypi.org/project/gpt-index/.', 'Twitter: https://twitter.com/gpt_index', 'Discord https://discord.gg/dGcwcsnxhU', 'LLMs are a phenomenonal piece of technology for knowledge generation and reasoning. They are pre-trained on large amounts of publicly available data.', 'How do we best augment LLMs with our own private data?', 'One paradigm that has emerged is in-context learning (the other is finetuning), where we insert context into the input prompt. That way, we take advantage of the LLM’s reasoning capabilities to generate a response.', 'To perform LLM’s data augmentation in a performant, efficient, and cheap manner, we need to solve two components:', 'D

In [24]:
from bs4 import BeautifulSoup
import urllib.parse
import html
import re
import requests

res = requests.get("https://gpt-index.readthedocs.io/en/latest/")
domain = "https://gpt-index.readthedocs.io/en/latest/"
domain_full = domain
soup = BeautifulSoup(res.text, 'html.parser')

# Find all links to local pages on the website
local_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.startswith(domain) or href.startswith('./') \
        or href.startswith('/') or href.startswith('modules') \
        or href.startswith('use_cases'):
        local_links.append(urllib.parse.urljoin(domain_full, href))

# Find the main content using CSS selectors
main_content = soup.select('body')[0]

# Extract the HTML code of the main content
main_content_html = str(main_content)

# Extract the plaintext of the main content
main_content_text = main_content.get_text()

# Remove all HTML tags
main_content_text = re.sub(r'<[^>]+>', '', main_content_text)

# Remove extra white space
main_content_text = ' '.join(main_content_text.split())

# Replace HTML entities with their corresponding characters
main_content_text = html.unescape(main_content_text)

print(main_content_text)

Contents Menu Expand Light mode Dark mode Auto light/dark mode Hide navigation sidebar Hide table of contents sidebar Toggle site navigation sidebar LlamaIndex Toggle Light / Dark / Auto color theme Toggle table of contents sidebar LlamaIndex Getting Started Installation and Setup Starter Tutorial Guides A Primer to using LlamaIndexToggle child pages in navigation LlamaIndex Usage Pattern How Each Index Works TutorialsToggle child pages in navigation 💬🤖 How to Build a Chatbot A Guide to Building a Full-Stack Web App with LLamaIndex A Guide to LlamaIndex + Structured Data A Guide to Extracting Terms and Definitions A Guide to Creating a Unified Query Framework over your Indexes SEC 10k Analysis Notebooks Use Cases Queries over your Data Integrations into LLM Applications Key Components Data Connectors (LlamaHub 🦙) Index StructuresToggle child pages in navigation How Each Index Works Updating an Index Composability Query InterfaceToggle child pages in navigation LlamaIndex Usage Pattern 

In [29]:
def scrape(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to scrape {url}")
        return None
    soup = BeautifulSoup(response.content, 'html.parser')
    main_content = soup.select_one('.document .body .section')
    if main_content is None:
        print(f"No content found in {url}")
        return None
    page_content = main_content.get_text()
    local_links = [link.get('href') for link in main_content.find_all('a') if link.get('href').startswith('/')]
    local_links = [urljoin(response.url, link) for link in local_links]
    return page_content, local_links


In [ ]:

data[3]

In [31]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record['url']
    } for i in range(len(texts))])

In [2]:
import os
import openai


openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

In [3]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [37]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

In [38]:
len(res['data'])

2

In [39]:

len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

In [58]:
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("22e6337a-1ab8-49c2-9ac4-281f09e6bde5") 
# find your environment next to the api key in pinecone console
env = os.getenv("us-west1-gcp") 


In [5]:
import pinecone

pinecone.init(
    api_key="22e6337a-1ab8-49c2-9ac4-281f09e6bde5",  # app.pinecone.io
    environment="us-west1-gcp"  # next to API key in console
)

index_name = "globaltelbot"

if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

index = pinecone.Index(index_name)

/home/cloudsuperadmin/.local/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

In [10]:
query = "can I use llamaindex with langchain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [ ]:

res

In [12]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [13]:
print(augmented_query)

LLM Application. Some of these applications are described below. Chatbots Chatbots are an incredibly popular use case for LLM’s. LlamaIndex gives you the tools to build Knowledge-augmented chatbots and agents. Relevant Resources: Building a Chatbot Using with a LangChain Agent Full-Stack Web Application LlamaIndex can be integrated into a downstream full-stack web application. It can be used in a backend server (such as Flask), packaged into a Docker container, and/or directly used in a framework such as Streamlit. We provide tutorials and resources to help you get started in this area. Relevant Resources: Fullstack Application Guide LlamaIndex Starter Pack Next Data Connectors (LlamaHub 🦙) Previous Queries over your Data Copyright © 2022, Jerry Liu Made with Sphinx and @pradyunsg's Furo On this page Integrations into LLM Applications Chatbots Full-Stack Web Application

---

connect your LLM’s with external data. Github: https://github.com/jerryjliu/llama_index PyPi: LlamaIndex: htt

In [14]:
# system message 
template = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": template},
        {"role": "user", "content": augmented_query}
    ]
)

In [15]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

Yes, according to the information provided, LlamaIndex can be used to build Knowledge-augmented chatbots and agents using LangChain. The documentation also includes a tutorial on how to build a chatbot with LangChain.